<h1>Integrantes<h1/>
<html>
<table border="1px">
<tr>
<td bgcolor="Aquamarine">Nome do Aluno</td>
<td bgcolor="Aquamarine">RA</td>
</tr>
<tr>
<td bgcolor="white">Gabriel Kury Fonseca</td>
<td bgcolor="white">10390103</td>
</tr>
<tr>
<td bgcolor="white">Luiz Gabriel Profirio Mendes</td>
<td bgcolor="white">10382703</td>
</tr>
    <tr>
<td bgcolor="white">Rafael de Toledo Navarro</td>
<td bgcolor="white">10389955</td>
</tr>
    </table>
</html>

<h2>Campo minado abaixo<h2\>

In [1]:
import numpy as np
import random

ROWS = 8
COLS = 10
MINES = 10

HIDDEN = '-'
MINE = '*'
FLAG = 'F'

# Função para criar o tabuleiro
def create_board():
    return [[HIDDEN for _ in range(COLS)] for _ in range(ROWS)]

# Função para plantar minas no tabuleiro
def place_mines(board, exclude_row, exclude_col):
    mine_positions = set()
    while len(mine_positions) < MINES:
        row = random.randint(0, ROWS - 1)
        col = random.randint(0, COLS - 1)
        if (row != exclude_row and col != exclude_col):
            mine_positions.add((row, col))
            board[row][col] = MINE
    return board

# Função para contar minas adjacentes a uma célula
def count_adjacent_mines(board, row, col):
    adjacent_mines = 0
    for i in range(max(0, row - 1), min(ROWS, row + 2)):
        for j in range(max(0, col - 1), min(COLS, col + 2)):
            if board[i][j] == MINE:
                adjacent_mines += 1
    return adjacent_mines

# Função para mostrar o tabuleiro
def print_board(board, reveal=False):
    for row in board:
        print(' '.join(row))
    print()

# Função para revelar a célula clicada
def reveal(board, visible_board, row, col):
    if board[row][col] == MINE:
        return False  # O jogador clicou em uma mina, perdeu o jogo
    if visible_board[row][col] != HIDDEN:
        return True  # Já revelado

    mines_around = count_adjacent_mines(board, row, col)
    visible_board[row][col] = str(mines_around)

    if mines_around == 0:
        for i in range(max(0, row - 1), min(ROWS, row + 2)):
            for j in range(max(0, col - 1), min(COLS, col + 2)):
                if visible_board[i][j] == HIDDEN:
                    reveal(board, visible_board, i, j)
    return True

# Função para verificar vitória
def check_victory(board, visible_board):
    for row in range(ROWS):
        for col in range(COLS):
            if visible_board[row][col] == HIDDEN and board[row][col] != MINE:
                return False
    return True

class MinesweeperAgent:
    def __init__(self):
        self.board = None
        self.visible_board = create_board()
        self.mines_placed = False

    def reset(self, first_click_row, first_click_col):
        self.board = create_board()
        self.board = place_mines(self.board, first_click_row, first_click_col)
        self.visible_board = create_board()
        self.mines_placed = True

    def get_state(self):
        # Representa o tabuleiro como uma matriz de números
        state = np.zeros((ROWS, COLS), dtype=float)

        for row in range(ROWS):
            for col in range(COLS):
                if self.visible_board[row][col] == HIDDEN:
                    state[row][col] = -1  # Ainda não revelado
                elif self.visible_board[row][col] == MINE:
                    state[row][col] = -2  # Minas
                else:
                    state[row][col] = float(self.visible_board[row][col])  # Número de minas adjacentes
        return state

    def get_action(self):
        state = self.get_state()

        # Encontre células não reveladas
        unsolved_cells = [(r, c) for r in range(ROWS) for c in range(COLS) if self.visible_board[r][c] == HIDDEN]

        # Estratégia simples de escolher aleatoriamente entre as células não reveladas
        action = random.choice(unsolved_cells)
        return action

    def step(self, action):
        row, col = action

        # Verifica se a ação é inválida
        if not (0 <= row < ROWS and 0 <= col < COLS):
            print(f"Ação inválida: célula ({row}, {col}) está fora do tabuleiro.")
            return self.get_state(), False  # Não finaliza o jogo, mas registra a invalidez

        if self.visible_board[row][col] != HIDDEN:
            print(f"Ação inválida: célula ({row}, {col}) já foi revelada ou marcada.")
            return self.get_state(), False  # Não finaliza o jogo

        # Revele a célula escolhida
        if not reveal(self.board, self.visible_board, row, col):
            return self.get_state(), True  # Game Over

        # Verifique se o agente ganhou
        if check_victory(self.board, self.visible_board):
            return self.get_state(), True  # Vitória

        return self.get_state(), False  # Ainda em jogo


# Exemplo de uso
agent = MinesweeperAgent()
agent.reset(0, 0)  # Coloca as minas com a primeira jogada na célula (0, 0)

# Simula o jogo
while True:
    state = agent.get_state()
    action = agent.get_action()
    print(f"Ação: Revelar {action}")
    next_state, done = agent.step(action)
    print_board(agent.visible_board)

    if done:
        print("Fim de jogo!")
        break


Ação: Revelar (7, 2)
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - 2 - - - - - - -

Ação: Revelar (5, 2)
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - - - - - - - - -
- - 2 - - - - - - -

Fim de jogo!


<h2>Modelo de IA Abaixo<h2\>

In [2]:
import tensorflow as tf
from collections import deque

In [3]:
class DeepQNetwork(tf.keras.Model):
    def __init__(self, input_shape, action_size):
        super(DeepQNetwork, self).__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape)
        self.dense2 = tf.keras.layers.Dense(128, activation='relu')
        self.output_layer = tf.keras.layers.Dense(action_size, activation='linear')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.output_layer(x)


In [4]:
class ReplayBuffer:
    def __init__(self, max_size):
        self.buffer = deque(maxlen=max_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def size(self):
        return len(self.buffer)


In [5]:
class DQAgent:
    def __init__(self, state_shape, action_size, lr=0.001, gamma=0.99, epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.1):
        self.state_shape = state_shape
        self.action_size = action_size
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.model = DeepQNetwork(state_shape, action_size)
        self.target_model = DeepQNetwork(state_shape, action_size)
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='mse')
        self.update_target_model()
        self.replay_buffer = ReplayBuffer(max_size=2000)

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def select_action(self, state):
        if np.random.rand() < self.epsilon:
            return random.randint(0, self.action_size - 1)
        q_values = self.model.predict(state[np.newaxis])
        return np.argmax(q_values[0])

    def train(self, batch_size):
        if self.replay_buffer.size() < batch_size:
            return
        minibatch = self.replay_buffer.sample(batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state[np.newaxis])
            if done:
                target[0][action] = reward
            else:
                next_q_values = self.target_model.predict(next_state[np.newaxis])
                target[0][action] = reward + self.gamma * np.max(next_q_values[0])
            self.model.train_on_batch(state[np.newaxis], target)
        self.epsilon = max(self.epsilon * self.epsilon_decay, self.epsilon_min)


In [6]:
agent = DQAgent(state_shape=(ROWS, COLS), action_size=ROWS * COLS)

for episode in range(100000):  # Número de episódios de treinamento
    minesweeper = MinesweeperAgent()
    minesweeper.reset(0, 0)
    state = minesweeper.get_state()
    print("\n\n New ep \n\n")
    max_steps = 1000  # Limite de passos por episódio
    step_count = 0
    while True:
        action_idx = agent.select_action(state.flatten())
        action = divmod(action_idx, COLS)
        next_state, done = minesweeper.step(action)
        reward = -10 if minesweeper.board[action[0]][action[1]] == MINE else 1

        # Penaliza ações inválidas
        if not (0 <= action[0] < ROWS and 0 <= action[1] < COLS) or minesweeper.visible_board[action[0]][action[1]] != HIDDEN:
            reward = -5  # Penalidade para ações inválidas

        if check_victory(minesweeper.board, minesweeper.visible_board):
            reward = 10
        agent.replay_buffer.add((state.flatten(), action_idx, reward, next_state.flatten(), done))
        state = next_state
        step_count += 1
        if done or step_count >= max_steps:  # Condição de saída
            break


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


A saída de streaming foi truncada nas últimas 5000 linhas.
Ação inválida: célula (4, 8) já foi revelada ou marcada.
Ação inválida: célula (4, 9) já foi revelada ou marcada.
Ação inválida: célula (0, 2) já foi revelada ou marcada.
Ação inválida: célula (6, 1) já foi revelada ou marcada.
Ação inválida: célula (3, 6) já foi revelada ou marcada.


 New ep 




 New ep 




 New ep 




 New ep 




 New ep 


Ação inválida: célula (2, 0) já foi revelada ou marcada.
Ação inválida: célula (1, 3) já foi revelada ou marcada.
Ação inválida: célula (7, 3) já foi revelada ou marcada.


 New ep 




 New ep 




 New ep 


Ação inválida: célula (1, 2) já foi revelada ou marcada.
Ação inválida: célula (7, 4) já foi revelada ou marcada.


 New ep 


Ação inválida: célula (2, 9) já foi revelada ou marcada.
Ação inválida: célula (4, 8) já foi revelada ou marcada.


 New ep 




 New ep 




 New ep 


Ação inválida: célula (2, 2) já foi revelada ou marcada.
Ação inválida: célula (5, 0) já foi revelada

<h2>Testes do modelo abaixo<h2/>

In [7]:
agent.epsilon = 0  # Desativa a exploração
wins = 0
losses = 0
num_tests = 200  # Número de jogos de teste

for test in range(num_tests):
    minesweeper = MinesweeperAgent()
    minesweeper.reset(0, 0)
    state = minesweeper.get_state()

    print(f"\n\nTESTE {test + 1}\n\n")

    max_steps = 1000  # Limite de passos por episódio
    step_count = 0
    while True:
        action_idx = agent.select_action(state.flatten())

        # Verifica ação inválida
        if not (0 <= action_idx < ROWS * COLS):
            print(f"Ação inválida: {action_idx}")
            losses += 1
            break

        action = divmod(action_idx, COLS)
        next_state, done = minesweeper.step(action)

        step_count += 1
        if done:
            if check_victory(minesweeper.board, minesweeper.visible_board):
                wins += 1
            else:
                losses += 1
            break

        # Verifica limite de passos
        if step_count >= max_steps:
            print(f"Limite de passos atingido no teste {test + 1}")
            losses += 1
            break

        # Verifica mudança de estado
        if (next_state == state).all():
            print("Estado não mudou, possível loop infinito.")
            losses += 1
            break

        state = next_state

print(f"Resultados após o treinamento:")
print(f"Vitórias: {wins}/{num_tests}")
print(f"Derrotas: {losses}/{num_tests}")
print(f"Taxa de vitória: {wins / num_tests * 100:.2f}%")




TESTE 1


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Ação inválida: célula (5, 2) já foi revelada ou marcada.
Estado não mudou, possível loop infinito.


TESTE 2


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Ação inválida: célula (7, 0) já foi revelada ou marcada.
Estado não mudou, possível loop infinito.


TESTE 3


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Ação inválida: célula (7, 3) já foi revelada ou marcada.
Estado não mudou, possível loop infinito.


TESTE 4


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Ação inválida: célula (5, 2) já foi revelada ou marcada.
Estado não mudou, possível loop infinito.


TESTE 5


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Ação inválida: célula (2, 6) já foi revelada ou m

In [8]:
agent.model.save("minesweeper_dqn_model.h5")